# Setup - Install Libraries

In [ ]:
# Run the following commands once, in order to install libraries - DO NOT Uncomment this line.

# Uncomment below lines

# !pip3 install --upgrade pip
# !pip3 install google-cloud-bigquery
# !pip3 install pandas-gbq -U
# !pip3 install db-dtypes
# !pip3 install packaging --upgrade

# Import libraries

In [5]:
# Import libraries
from google.cloud import bigquery
import pandas as pd
from pandas_gbq import to_gbq
import os

print('Libraries imported successfully')

Libraries imported successfully


In [6]:
# Set the environment variable for Google Cloud credentials
# Place the path in which the .json file is located.

# Example (if .json is located in the same directory with the notebook)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "at-arch-416714-6f9900ec7.json"

# -- YOUR CODE GOES BELOW THIS LINE
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/nikolas/Documents/_Επιμόρφωση/_Data_Analytics_Modern_Corporate_Business/Capstone_Project_Resources/studied-brand-481913-u8-0f4005891f76.json" # Edit path
# -- YOUR CODE GOES ABOVE THIS LINE

In [7]:
# Set your Google Cloud project ID and BigQuery dataset details

# -- YOUR CODE GOES BELOW THIS

project_id = 'studied-brand-481913-u8' # Edit with your project id
dataset_id = 'staging_db' # Modify the necessary schema name: staging_db, reporting_db etc.
table_id = 'stg_address' # Modify the necessary table name: stg_customer, stg_city etc.

# -- YOUR CODE GOES ABOVE THIS LINE

# SQL Query

In [8]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

# -- YOUR CODE GOES BELOW THIS LINE

# Define your SQL query here
query = """
with base as (
  select *
  from `studied-brand-481913-u8.postgres_productionpublic.address` --Your table path
  )

  , final as (
    select
        address_id
        , address as address_address
        , address2 as address_address2
        , district as address_district
        , city_id as address_city_id
        , postal_code as address_postal_code
        , phone as address_phone
        , last_update as address_last_update  
   FROM base
  )

  select * from final
"""

# -- YOUR CODE GOES ABOVE THIS LINE

# Execute the query and store the result in a dataframe
df = client.query(query).to_dataframe()

# Explore some records
df.head()

/Users/nikolas/Library/Python/3.11/lib/python/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,address_id,address_address,address_address2,address_district,address_city_id,address_postal_code,address_phone,address_last_update
0,21,270 Toulon Boulevard,,Kalmykia,156,81766,407752414682,2022-02-15 09:45:30+00:00
1,52,909 Garland Manor,,Tatarstan,367,69367,705800322606,2022-02-15 09:45:30+00:00
2,10,1795 Santiago de Compostela Way,,Texas,295,18743,860452626434,2022-02-15 09:45:30+00:00
3,2,28 MySQL Boulevard,None,QLD,576,,,2022-02-15 09:45:30+00:00
4,12,478 Joliet Way,,Hamilton,200,77948,657282285970,2022-02-15 09:45:30+00:00


# Write to BigQuery

In [9]:
# Define the full table ID
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# -- YOUR CODE GOES BELOW THIS LINE
# Define table schema based on the project description

schema = [
    bigquery.SchemaField('address_id', 'INTEGER'),
    bigquery.SchemaField('address_address', 'STRING'),
    bigquery.SchemaField('address_address2', 'STRING'),
    bigquery.SchemaField('address_district', 'STRING'),
    bigquery.SchemaField('address_city_id', 'INTEGER'),
    bigquery.SchemaField('address_postal_code', 'STRING'),
    bigquery.SchemaField('address_phone', 'STRING'),
    bigquery.SchemaField('address_last_update', 'DATETIME'),
    ]

# -- YOUR CODE GOES ABOVE THIS LINE

In [10]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

# Check if the table exists
def table_exists(client, full_table_id):
    try:
        client.get_table(full_table_id)
        return True
    except Exception:
        return False

# Write the dataframe to the table (overwrite if it exists, create if it doesn't)
if table_exists(client, full_table_id):
    # If the table exists, overwrite it
    destination_table = f"{dataset_id}.{table_id}"
    # Write the dataframe to the table (overwrite if it exists)
    to_gbq(df, destination_table, project_id=project_id, if_exists='replace')
    print(f"Table {full_table_id} exists. Overwritten.")
else:
    # If the table does not exist, create it
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
    job.result()  # Wait for the job to complete
    print(f"Table {full_table_id} did not exist. Created and data loaded.")

Table studied-brand-481913-u8.staging_db.stg_address did not exist. Created and data loaded.


In [12]:
# Below line converts your i.pynb file to .py python executable file. Modify the input and output names based
# on the table you are processing.
# Example:
# ! jupyter nbconvert stg_customer.ipynb --to python

# -- YOUR CODE GOES BELOW THIS LINE

!python3 -m jupyter nbconvert stg_address.ipynb --to python

# -- YOUR CODE GOES ABOVE THIS LINE

[NbConvertApp] Converting notebook stg_address.ipynb to python
[NbConvertApp] Writing 4440 bytes to stg_address.py
